In [2]:
#Group project DSCI 100 Group #3

# Testing if github collaboration works

library(tidyverse)
library(repr)
library(tidymodels)
options(repr.matrix.max.rows = 6)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.6     ✔ purrr   0.3.4
✔ tibble  3.1.7     ✔ dplyr   1.0.9
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.1.2     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.0     ✔ rsample      1.0.0
✔ dials        1.0.0     ✔ tune         1.0.0
✔ infer        1.0.2     ✔ workflows    1.0.0
✔ modeldata    1.0.0     ✔ workflowsets 1.0.0
✔ parsnip      1.0.0     ✔ yardstick    1.0.0
✔ recipes      1.0.1     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks r

*choosing audit data set from here: https://archive.ics.uci.edu/ml/datasets/Audit+Data

Title:

Main question: 

Dataset of interest: 

Preliminary exploratory data analysis:
the cell below demonstrates reading our data into R.


In [13]:
audit_data <- read.csv("audit_risk.csv")
audit_data
## clean data (use tidy data criteria); 
# each row is a single observation,
# each column is a single variable, and
# each value is a single cell (i.e., its entry in the data frame is not shared with another value).

##tidying data:
(audit_data_tidy <- select(audit_data, Sector_score, LOCATION_ID, Inherent_Risk, CONTROL_RISK, Detection_Risk, Audit_Risk))

Sector_score,LOCATION_ID,PARA_A,Score_A,Risk_A,PARA_B,Score_B,Risk_B,TOTAL,numbers,⋯,RiSk_E,History,Prob,Risk_F,Score,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk,Risk
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>
3.89,23,4.18,0.6,2.508,2.50,0.2,0.500,6.68,5,⋯,0.4,0,0.2,0,2.4,8.574,0.4,0.5,1.7148,1
3.89,6,0.00,0.2,0.000,4.83,0.2,0.966,4.83,5,⋯,0.4,0,0.2,0,2.0,2.554,0.4,0.5,0.5108,0
3.89,6,0.51,0.2,0.102,0.23,0.2,0.046,0.74,5,⋯,0.4,0,0.2,0,2.0,1.548,0.4,0.5,0.3096,0
⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋱,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮,⋮
55.57,14,0.24,0.2,0.048,0.04,0.2,0.008,0.28,5,⋯,0.4,0,0.2,0,2,1.456,0.4,0.5,0.2912,0
55.57,18,0.20,0.2,0.040,0.00,0.2,0.000,0.20,5,⋯,0.4,0,0.2,0,2,1.440,0.4,0.5,0.2880,0
55.57,15,0.00,0.2,0.000,0.00,0.2,0.000,0.00,5,⋯,0.4,0,0.2,0,2,1.464,0.4,0.5,0.2928,0


Sector_score,LOCATION_ID,Inherent_Risk,CONTROL_RISK,Detection_Risk,Audit_Risk
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
3.89,23,8.574,0.4,0.5,1.7148
3.89,6,2.554,0.4,0.5,0.5108
3.89,6,1.548,0.4,0.5,0.3096
⋮,⋮,⋮,⋮,⋮,⋮
55.57,14,1.456,0.4,0.5,0.2912
55.57,18,1.440,0.4,0.5,0.2880
55.57,15,1.464,0.4,0.5,0.2928


In [12]:
audit_split <- initial_split(audit_data_tidy, prop = 0.75, strata = Audit_Risk)

audit_training <- training(audit_split)
audit_testing <- testing(audit_split)

Methods:

We will use xyz column to analyze our data. In particular, we will perform a classification to predict a category for a new observation using past observations of known class. In particular, we will predict xyz based on past observations of described class of xyz. 

Expected outcomes and significance:

What do you expect to find?
What impact could such findings have?
What future questions could this lead to?